In [5]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [3]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [8]:
# Create a list for holding the cities
cities = []
# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count
len(cities)

623

In [12]:
# Import the requests library
import requests

# Import the API key
from config import weather_api_key

In [13]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [10]:
# Import the time library and the datetime module from the datetime library
import time
from datetime import datetime

In [14]:
# Create an empty list to hold the weather data
city_data = []
# Print the beginning of the logging
print("Beginning Data Retrieval      ")
print("------------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all cities in our list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ", "+")

    # Log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval      
------------------------------
Processing Record 1 of Set 1 | campohermoso
Processing Record 2 of Set 1 | shellbrook
Processing Record 3 of Set 1 | kodiak
Processing Record 4 of Set 1 | arlit
Processing Record 5 of Set 1 | pevek
Processing Record 6 of Set 1 | bredasdorp
Processing Record 7 of Set 1 | rongcheng
Processing Record 8 of Set 1 | vao
Processing Record 9 of Set 1 | hermanus
Processing Record 10 of Set 1 | tautira
Processing Record 11 of Set 1 | nador
Processing Record 12 of Set 1 | torbay
Processing Record 13 of Set 1 | rikitea
Processing Record 14 of Set 1 | sentyabrskiy
City not found. Skipping...
Processing Record 15 of Set 1 | kapaa
Processing Record 16 of Set 1 | punta arenas
Processing Record 17 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 18 of Set 1 | kaeo
Processing Record 19 of Set 1 | bethel
Processing Record 20 of Set 1 | leh
Processing Record 21 of Set 1 | ballina
Processing Record 22 of Set 1 | grand river 

In [15]:
# Convert the array of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Campohermoso,36.9357,-2.1337,56.12,74,0,2.59,ES,2022-02-28 19:54:39
1,Shellbrook,53.2168,-106.4011,7.83,85,100,0.94,CA,2022-02-28 19:54:40
2,Kodiak,57.7900,-152.4072,37.02,81,68,7.18,US,2022-02-28 19:54:40
3,Arlit,18.7369,7.3853,79.47,9,0,5.86,NE,2022-02-28 19:54:40
4,Pevek,69.7008,170.3133,3.45,95,100,5.79,RU,2022-02-28 19:54:41


In [19]:
# Reorder the columns
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]

city_data_df = city_data_df[new_column_order]

city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Campohermoso,ES,2022-02-28 19:54:39,36.9357,-2.1337,56.12,74,0,2.59
1,Shellbrook,CA,2022-02-28 19:54:40,53.2168,-106.4011,7.83,85,100,0.94
2,Kodiak,US,2022-02-28 19:54:40,57.7900,-152.4072,37.02,81,68,7.18
3,Arlit,NE,2022-02-28 19:54:40,18.7369,7.3853,79.47,9,0,5.86
4,Pevek,RU,2022-02-28 19:54:41,69.7008,170.3133,3.45,95,100,5.79


In [20]:
# Create the output file (CSV)
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV
city_data_df.to_csv(output_data_file, index="City_ID")